In [ ]:
from typing import Set, List, Dict
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
!ls

In [ ]:
demands_deliveries_file = pd.read_csv("data-demands-deliveyconstraint.csv")
routes_file = pd.read_csv("routes.csv")

In [ ]:
# demands_dict = demands_deliveries_file[["Stores", "Demands (per week)"]].set_index('Stores').to_dict()["Demands (per week)"]

In [ ]:
# Ordered with idx 0: 'S1', 1: 'S2' etc...
stores: Dict[int, str] = demands_deliveries_file["Demands (per week)"].to_dict()

n_stores = len(stores)

In [ ]:
# Constants
n_days = 7

In [ ]:

# Create a new model
model: Model = Model("transportation")

# Create variables Xijd where i indicates first store, j second, d the day
variables = model.addVars(n_stores,n_stores, n_days)


# Set objective. Since cost is a constant we can simply minimize the units
model.setObjective(sum(variables[i,j, d]
                    for i in range(n_stores)
                    for j in range(n_stores)
                    for d in range(n_days)), GRB.MINIMIZE)



# Add constraint for max capacity:
for store_i in stores.keys():
    for store_j in stores.keys():
        for day in range(n_days):
            model.addConstr(variables[store_i, store_j, day] <= 20)


# Each truck cannot carry negative units
for store_i in stores.keys():
    for store_j in stores.keys():
        for day in range(n_days):
            model.addConstr(variables[store_i, store_j, day] >= 0)


# Add demand meeting constraint
for store_i, store_i_demands in stores.items():
            model.addConstr(
            sum(variables[store_i, store_j, day]
            for store_j in stores.keys()
            for day in range(n_days)) >= store_i_demands)

model.optimize()

# print optimal solutions
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))

# print optimal value
print('Obj: %g' % model.objVal)

# print dual values to all constraints
print(model.getAttr("Pi", model.getConstrs()))